In [1]:
import lightning as L 
import matplotlib.pyplot as plt
import lightning as L
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor, EarlyStopping, StochasticWeightAveraging
import torch 

torch.set_float32_matmul_precision("high")

In [2]:
from src.models.SimpleFC import SimpleFC_Lit
from src.models.FCSplit import FC_Split

In [3]:
import numpy as np

In [5]:
for pl_i in range(9):
    model = SimpleFC_Lit.load_from_checkpoint("lightning_logs/FullCNN_WeightedLoss_Large_ActivationTest/version_{}/checkpoints/last.ckpt".format(pl_i))
    model.setup()
    device = "cpu"
    model.to(device)
    model.eval()
    ind_list = [-1, -2, -3, -1501,-1506,-1510,-1511,-1514,-1515, -1517, -1521]

    fig,ax = plt.subplots(len(ind_list),2, figsize=(24,8*len(ind_list)))
    for i,ind in enumerate(ind_list):
        x_test, y_test = model.test_dataset[ind]
        x_test = x_test.to(device).reshape(1,-1)
        y_pred = model(x_test).detach().numpy()

        if i == 0:
            text = "HyperParams:\nActivation:{}\nBatchNorm:{}\nDropout:{}".format(model.hparams["activation"],model.hparams["with_batchnorm"],model.hparams["dropout"])
            ax[i,1].text(0.85, 0.05, text, transform=ax[i,1].transAxes, fontsize=14,
            verticalalignment='bottom')
        ax[i,0].plot(y_pred[0,100:], marker="x", label="im, pred (n="+str(x_test.detach().numpy()[0,0])+")")
        ax[i,0].plot(y_test[100:], marker="x", label="im, true")
        ax[i,0].legend()

        ax[i,1].plot(y_pred[0,0:100], marker="x", label="re, pred")
        ax[i,1].plot(y_test[0:100], marker="x", label="re, true")
        ax[i,1].legend()
        
    #fig.suptitle('No Dropout, Weighted, Medium Model Size, Medium Training Size ', fontsize=20)
    plt.savefig("demo_{}.pdf".format(pl_i))
    #plt.show()